# Latency Analysis Dashboard

This notebook analyzes the performance metrics from the FunctionGemma Agent logs:
- Token generation latency
- Tool execution time
- Request patterns
- Performance bottlenecks

In [ ]:
# Install required packages
# !pip install pandas matplotlib seaborn plotly ipywidgets

import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta
import ipywidgets as widgets
from IPython.display import display, HTML
import warnings
warnings.filterwarnings('ignore')

# Configure plotting
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

# Configure pandas display
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

## 1. Load and Parse Log Data

In [ ]:
def parse_json_logs(log_file_path):
    """Parse JSON logs into a DataFrame."""
    logs = []
    
    try:
        with open(log_file_path, 'r') as f:
            for line in f:
                line = line.strip()
                if line:
                    try:
                        log_entry = json.loads(line)
                        logs.append(log_entry)
                    except json.JSONDecodeError:
                        # Skip malformed lines
                        continue
    
    df = pd.DataFrame(logs)
    
    # Convert timestamp
    if 'timestamp' in df.columns:
        df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    return df

# Load the logs
log_file = "../logs/json_logs.log"  # Update path as needed

# For demo, create sample data if file doesn't exist
import os
if not os.path.exists(log_file):
    print("Creating sample log data for demonstration...")
    create_sample_logs(log_file)

# Parse logs
df = parse_json_logs(log_file)

print(f"Loaded {len(df)} log entries")
print(f"Time range: {df['timestamp'].min()} to {df['timestamp'].max()}")

# Display sample data
print("\nSample log entries:")
display(df.head())

In [ ]:
def create_sample_logs(filename):
    """Create sample log data for demonstration."""
    import random
    
    # Generate sample data
    num_entries = 1000
    base_time = datetime.now() - timedelta(hours=24)
    
    logs = []
    for i in range(num_entries):
        timestamp = base_time + timedelta(seconds=i * 86.4)  # Spread over 24 hours
        
        # Random latency values
        token_latency = np.random.normal(100, 30)  # Mean 100ms, std 30ms
        tool_latency = np.random.exponential(50)  # Mean 50ms
        
        log_entry = {
            "timestamp": timestamp.isoformat(),
            "request_id": f"req-{i:06d}",
            "token_latency_ms": max(10, token_latency),
            "tool_execution_ms": max(5, tool_latency),
            "total_latency_ms": max(20, token_latency + tool_latency + np.random.normal(20, 5)),
            "tokens_used": random.randint(50, 500),
            "tools_used": random.randint(0, 3),
            "status": random.choice(["success", "success", "success", "error"]),
            "model_version": "gemma-270m-v1.0"
        }
        logs.append(log_entry)
    
    # Write to file
    os.makedirs(os.path.dirname(filename), exist_ok=True)
    with open(filename, 'w') as f:
        for log in logs:
            f.write(json.dumps(log) + '\n')
    
    print(f"Created {num_entries} sample log entries in {filename}")

## 2. Overall Performance Overview

In [ ]:
# Calculate key metrics
metrics = {
    'Total Requests': len(df),
    'Success Rate': f"{(df['status'] == 'success').mean() * 100:.1f}%",
    'Avg Total Latency': f"{df['total_latency_ms'].mean():.2f}ms",
    'P95 Total Latency': f"{df['total_latency_ms'].quantile(0.95):.2f}ms",
    'Avg Token Latency': f"{df['token_latency_ms'].mean():.2f}ms",
    'Avg Tool Latency': f"{df['tool_execution_ms'].mean():.2f}ms",
    'Avg Tokens per Request': f"{df['tokens_used'].mean():.0f}",
    'Requests per Hour': f"{len(df) / 24:.0f}"
}

# Display metrics
print("\n📊 Performance Summary")
print("=" * 50)
for metric, value in metrics.items():
    print(f"{metric:.<30} {value}")

# Create a summary dashboard
fig = make_subplots(
    rows=2, cols=2,
    subplot_titles=('Latency Distribution', 'Status Breakdown', 'Hourly Request Volume', 'Token Usage'),
    specs=[[{"secondary_y": False}, {"type": "domain"}],
            [{"secondary_y": False}, {"secondary_y": False}]]
)

# 1. Latency distribution histogram
fig.add_trace(
    go.Histogram(x=df['total_latency_ms'], name='Latency', nbinsx=50),
    row=1, col=1
)

# 2. Status pie chart
status_counts = df['status'].value_counts()
fig.add_trace(
    go.Pie(labels=status_counts.index, values=status_counts.values, name="Status"),
    row=1, col=2
)

# 3. Hourly request volume
df['hour'] = df['timestamp'].dt.hour
hourly_counts = df.groupby('hour').size().reset_index(name='count')
fig.add_trace(
    go.Bar(x=hourly_counts['hour'], y=hourly_counts['count'], name='Requests'),
    row=2, col=1
)

# 4. Token usage scatter
fig.add_trace(
    go.Scatter(
        x=df['tokens_used'],
        y=df['token_latency_ms'],
        mode='markers',
        name='Tokens vs Latency',
        opacity=0.6
    ),
    row=2, col=2
)

fig.update_layout(height=800, showlegend=False, title_text="Performance Overview Dashboard")
fig.show()

## 3. Token Generation Latency Analysis

In [ ]:
# Detailed token latency analysis
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Token latency histogram
axes[0, 0].hist(df['token_latency_ms'], bins=50, alpha=0.7, color='skyblue', edgecolor='black')
axes[0, 0].axvline(df['token_latency_ms'].mean(), color='red', linestyle='--', label=f"Mean: {df['token_latency_ms'].mean():.1f}ms")
axes[0, 0].axvline(df['token_latency_ms'].median(), color='green', linestyle='--', label=f"Median: {df['token_latency_ms'].median():.1f}ms")
axes[0, 0].set_title('Token Generation Latency Distribution')
axes[0, 0].set_xlabel('Latency (ms)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].legend()

# 2. Token latency over time
df_sorted = df.sort_values('timestamp')
axes[0, 1].plot(df_sorted['timestamp'], df_sorted['token_latency_ms'], alpha=0.5, linewidth=0.5)
# Add rolling average
rolling_avg = df_sorted['token_latency_ms'].rolling(window=50).mean()
axes[0, 1].plot(df_sorted['timestamp'], rolling_avg, color='red', linewidth=2, label='50-point MA')
axes[0, 1].set_title('Token Latency Over Time')
axes[0, 1].set_xlabel('Time')
axes[0, 1].set_ylabel('Latency (ms)')
axes[0, 1].legend()
axes[0, 1].tick_params(axis='x', rotation=45)

# 3. Latency percentiles
percentiles = [50, 75, 90, 95, 99]
latency_percentiles = [df['token_latency_ms'].quantile(p/100) for p in percentiles]
bars = axes[1, 0].bar([f'P{p}' for p in percentiles], latency_percentiles, color='lightcoral')
axes[1, 0].set_title('Token Latency Percentiles')
axes[1, 0].set_ylabel('Latency (ms)')
# Add value labels
for bar in bars:
    height = bar.get_height()
    axes[1, 0].text(bar.get_x() + bar.get_width()/2., height,
                   f'{height:.1f}ms', ha='center', va='bottom')

# 4. Latency vs Tokens Used
axes[1, 1].scatter(df['tokens_used'], df['token_latency_ms'], alpha=0.3, s=20)
# Add trend line
z = np.polyfit(df['tokens_used'], df['token_latency_ms'], 1)
p = np.poly1d(z)
axes[1, 1].plot(df['tokens_used'], p(df['tokens_used']), "r--", alpha=0.8)
axes[1, 1].set_title('Token Latency vs Tokens Generated')
axes[1, 1].set_xlabel('Tokens Generated')
axes[1, 1].set_ylabel('Latency (ms)')

plt.tight_layout()
plt.show()

# Print statistics
print("\n🔍 Token Latency Statistics")
print("=" * 50)
print(f"Mean: {df['token_latency_ms'].mean():.2f}ms")
print(f"Median: {df['token_latency_ms'].median():.2f}ms")
print(f"Std Dev: {df['token_latency_ms'].std():.2f}ms")
print(f"Min: {df['token_latency_ms'].min():.2f}ms")
print(f"Max: {df['token_latency_ms'].max():.2f}ms")
print(f"P95: {df['token_latency_ms'].quantile(0.95):.2f}ms")
print(f"P99: {df['token_latency_ms'].quantile(0.99):.2f}ms")

## 4. Tool Execution Time Analysis

In [ ]:
# Filter requests that used tools
df_with_tools = df[df['tools_used'] > 0].copy()

print(f"\n🛠️ Tool Execution Analysis")
print("=" * 50)
print(f"Requests using tools: {len(df_with_tools)} ({len(df_with_tools)/len(df)*100:.1f}%)")
print(f"Average tools per request: {df_with_tools['tools_used'].mean():.2f}")

# Create visualizations
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# 1. Tool execution time distribution
axes[0, 0].hist(df['tool_execution_ms'], bins=50, alpha=0.7, color='lightgreen', edgecolor='black')
axes[0, 0].axvline(df['tool_execution_ms'].mean(), color='red', linestyle='--', 
                label=f"Mean: {df['tool_execution_ms'].mean():.1f}ms")
axes[0, 0].set_title('Tool Execution Time Distribution')
axes[0, 0].set_xlabel('Time (ms)')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].legend()

# 2. Tool count vs execution time
tool_time_stats = df.groupby('tools_used')['tool_execution_ms'].agg(['mean', 'std', 'count']).reset_index()
axes[0, 1].bar(tool_time_stats['tools_used'], tool_time_stats['mean'], 
               yerr=tool_time_stats['std'], capsize=5, color='orange', alpha=0.7)
axes[0, 1].set_title('Tool Execution Time by Number of Tools')
axes[0, 1].set_xlabel('Number of Tools Used')
axes[0, 1].set_ylabel('Average Execution Time (ms)')

# 3. Tool vs Token latency comparison
axes[1, 0].scatter(df['token_latency_ms'], df['tool_execution_ms'], alpha=0.3, s=20)
axes[1, 0].set_title('Token Latency vs Tool Execution Time')
axes[1, 0].set_xlabel('Token Latency (ms)')
axes[1, 0].set_ylabel('Tool Execution Time (ms)')

# 4. Contribution to total latency
df['token_contribution'] = df['token_latency_ms'] / df['total_latency_ms'] * 100
df['tool_contribution'] = df['tool_execution_ms'] / df['total_latency_ms'] * 100
df['other_contribution'] = 100 - df['token_contribution'] - df['tool_contribution']

contributions = [
    df['token_contribution'].mean(),
    df['tool_contribution'].mean(),
    df['other_contribution'].mean()
]
labels = ['Token Generation', 'Tool Execution', 'Other']
colors = ['skyblue', 'lightgreen', 'lightcoral']

axes[1, 1].pie(contributions, labels=labels, colors=colors, autopct='%1.1f%%', startangle=90)
axes[1, 1].set_title('Average Latency Contribution Breakdown')

plt.tight_layout()
plt.show()

# Tool execution statistics
print("\n📊 Tool Execution Statistics")
print("=" * 50)
print(f"Mean tool time: {df['tool_execution_ms'].mean():.2f}ms")
print(f"Max tool time: {df['tool_execution_ms'].max():.2f}ms")
print(f"P95 tool time: {df['tool_execution_ms'].quantile(0.95):.2f}ms")

## 5. Performance Patterns and Anomalies

In [ ]:
# Detect anomalies in latency
def detect_anomalies(data, threshold=2):
    """Detect anomalies using z-score threshold."""
    z_scores = np.abs((data - data.mean()) / data.std())
    return z_scores > threshold

# Find anomalies
latency_anomalies = detect_anomalies(df['total_latency_ms'])
anomaly_count = latency_anomalies.sum()

print(f"\n🚨 Anomaly Detection")
print("=" * 50)
print(f"Anomalies detected: {anomaly_count} ({anomaly_count/len(df)*100:.1f}% of requests)")

if anomaly_count > 0:
    print("\nTop 5 anomalies:")
    anomaly_df = df[latency_anomalies].nlargest(5, 'total_latency_ms')
    display(anomaly_df[['timestamp', 'total_latency_ms', 'token_latency_ms', 
                       'tool_execution_ms', 'tools_used', 'status']])

# Visualize anomalies
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# 1. Latency over time with anomalies highlighted
normal_mask = ~latency_anomalies
axes[0].scatter(df[normal_mask]['timestamp'], df[normal_mask]['total_latency_ms'], 
               alpha=0.5, s=10, label='Normal', color='blue')
axes[0].scatter(df[latency_anomalies]['timestamp'], df[latency_anomalies]['total_latency_ms'], 
               alpha=0.8, s=20, label='Anomaly', color='red')
axes[0].set_title('Latency Over Time (Anomalies Highlighted)')
axes[0].set_xlabel('Time')
axes[0].set_ylabel('Total Latency (ms)')
axes[0].legend()
axes[0].tick_params(axis='x', rotation=45)

# 2. Hourly performance pattern
df['hour'] = df['timestamp'].dt.hour
hourly_stats = df.groupby('hour')['total_latency_ms'].agg(['mean', 'std']).reset_index()

axes[1].errorbar(hourly_stats['hour'], hourly_stats['mean'], yerr=hourly_stats['std'], 
                 marker='o', capsize=5, capthick=2, color='purple')
axes[1].set_title('Average Latency by Hour of Day')
axes[1].set_xlabel('Hour of Day')
axes[1].set_ylabel('Average Latency (ms)')
axes[1].set_xticks(range(0, 24, 2))

plt.tight_layout()
plt.show()

## 6. Interactive Performance Explorer

In [ ]:
# Create interactive widgets for exploration
style = {'description_width': 'initial'}

# Time range selector
start_date = widgets.DatePicker(
    value=df['timestamp'].min().date(),
    description='Start Date:',
    style=style
)

end_date = widgets.DatePicker(
    value=df['timestamp'].max().date(),
    description='End Date:',
    style=style
)

# Metric selector
metric_dropdown = widgets.Dropdown(
    options=[
        ('Total Latency', 'total_latency_ms'),
        ('Token Latency', 'token_latency_ms'),
        ('Tool Execution Time', 'tool_execution_ms'),
        ('Tokens Used', 'tokens_used')
    ],
    value='total_latency_ms',
    description='Metric:',
    style=style
)

# Aggregation selector
agg_dropdown = widgets.Dropdown(
    options=[('Mean', 'mean'), ('Median', 'median'), ('P95', lambda x: x.quantile(0.95)), ('Count', 'count')],
    value='mean',
    description='Aggregation:',
    style=style
)

# Output widget
output = widgets.Output()

def update_analysis(_):
    with output:
        output.clear_output()
        
        # Filter data
        mask = (df['timestamp'].dt.date >= start_date.value) & \
               (df['timestamp'].dt.date <= end_date.value)
        filtered_df = df[mask]
        
        if len(filtered_df) == 0:
            print("No data in selected range")
            return
        
        # Apply aggregation
        if agg_dropdown.value == 'count':
            filtered_df = filtered_df.groupby(filtered_df['timestamp'].dt.floor('H')).size().reset_index(name='value')
            filtered_df.columns = ['timestamp', 'value']
            ylabel = 'Request Count'
        else:
            if callable(agg_dropdown.value):
                result = filtered_df.groupby(filtered_df['timestamp'].dt.floor('H'))[metric_dropdown.value].apply(agg_dropdown.value)
            else:
                result = filtered_df.groupby(filtered_df['timestamp'].dt.floor('H'))[metric_dropdown.value].agg(agg_dropdown.value)
            filtered_df = result.reset_index(name='value')
            ylabel = f'{metric_dropdown.value} ({agg_dropdown.value})'
        
        # Create plot
        fig = go.Figure()
        fig.add_trace(go.Scatter(
            x=filtered_df['timestamp'],
            y=filtered_df['value'],
            mode='lines+markers',
            name=metric_dropdown.value
        ))
        
        fig.update_layout(
            title=f'{metric_dropdown.value} Over Time',
            xaxis_title='Time',
            yaxis_title=ylabel,
            hovermode='x unified'
        )
        
        fig.show()
        
        # Print summary
        print(f"\n📈 Analysis Summary")
        print(f"Period: {start_date.value} to {end_date.value}")
        print(f"Total requests: {len(mask[mask])}")
        print(f"Average {metric_dropdown.value}: {df[mask][metric_dropdown.value].mean():.2f}")

# Create dashboard
dashboard = widgets.VBox([
    widgets.HBox([start_date, end_date]),
    widgets.HBox([metric_dropdown, agg_dropdown]),
    output
])

# Connect widgets
start_date.observe(update_analysis, names='value')
end_date.observe(update_analysis, names='value')
metric_dropdown.observe(update_analysis, names='value')
agg_dropdown.observe(update_analysis, names='value')

# Display dashboard
print("\n📊 Interactive Performance Dashboard")
print("=" * 50)
print("Use the controls above to explore performance metrics")
display(dashboard)

# Initial plot
update_analysis(None)

## 7. Performance Optimization Recommendations

In [ ]:
def generate_recommendations(df):
    """Generate performance optimization recommendations."""
    recommendations = []
    
    # Check overall latency
    p95_latency = df['total_latency_ms'].quantile(0.95)
    if p95_latency > 500:
        recommendations.append({
            'category': 'High Latency',
            'priority': 'High',
            'issue': f'P95 latency is {p95_latency:.0f}ms (>500ms threshold)',
            'recommendation': 'Consider enabling request batching or upgrading hardware'
        })
    
    # Check token generation efficiency
    tokens_per_ms = df['tokens_used'] / df['token_latency_ms']
    if tokens_per_ms.mean() < 1:
        recommendations.append({
            'category': 'Token Generation',
            'priority': 'Medium',
            'issue': f'Low token generation rate: {tokens_per_ms.mean():.2f} tokens/ms',
            'recommendation': 'Optimize model quantization or use GPU acceleration'
        })
    
    # Check tool execution
    avg_tool_time = df[df['tools_used'] > 0]['tool_execution_ms'].mean()
    if avg_tool_time > 100:
        recommendations.append({
            'category': 'Tool Performance',
            'priority': 'Medium',
            'issue': f'High tool execution time: {avg_tool_time:.0f}ms average',
            'recommendation': 'Optimize tool code or add caching for frequent operations'
        })
    
    # Check error rate
    error_rate = (df['status'] == 'error').mean() * 100
    if error_rate > 5:
        recommendations.append({
            'category': 'Reliability',
            'priority': 'High',
            'issue': f'High error rate: {error_rate:.1f}%',
            'recommendation': 'Investigate error patterns and implement better error handling'
        })
    
    # Check for patterns
    hourly_latency = df.groupby(df['timestamp'].dt.hour)['total_latency_ms'].mean()
    peak_hour = hourly_latency.idxmax()
    peak_latency = hourly_latency.max()
    avg_latency = hourly_latency.mean()
    
    if peak_latency > avg_latency * 1.5:
        recommendations.append({
            'category': 'Capacity Planning',
            'priority': 'Low',
            'issue': f'Peak latency at hour {peak_hour} is {peak_latency/avg_latency:.1f}x average',
            'recommendation': 'Consider auto-scaling or load balancing for peak hours'
        })
    
    return recommendations

# Generate and display recommendations
recommendations = generate_recommendations(df)

print("\n💡 Performance Optimization Recommendations")
print("=" * 60)

if recommendations:
    # Create DataFrame for better display
    rec_df = pd.DataFrame(recommendations)
    display(rec_df[['category', 'priority', 'issue', 'recommendation']])
    
    # Priority breakdown
    print("\n📋 Priority Breakdown:")
    priority_counts = rec_df['priority'].value_counts()
    for priority, count in priority_counts.items():
        print(f"  {priority}: {count} recommendation(s)")
else:
    print("✅ No major performance issues detected. System is performing well!")

# Create a summary visualization
if recommendations:
    fig, ax = plt.subplots(figsize=(10, 6))
    
    # Group by category
    category_counts = rec_df['category'].value_counts()
    
    # Create horizontal bar chart
    bars = ax.barh(category_counts.index, category_counts.values, color='steelblue')
    
    # Customize
    ax.set_title('Performance Issues by Category', fontweight='bold', fontsize=14)
    ax.set_xlabel('Number of Issues')
    
    # Add value labels
    for bar in bars:
        width = bar.get_width()
        ax.text(width + 0.1, bar.get_y() + bar.get_height()/2,
               f'{int(width)}', ha='left', va='center')
    
    plt.tight_layout()
    plt.show()

## 8. Export Report

In [ ]:
def generate_performance_report(df, recommendations):
    """Generate a comprehensive performance report."""
    
    report = f"""
# FunctionGemma Agent Performance Report
Generated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

## Executive Summary
- Total Requests Analyzed: {len(df):,}
- Analysis Period: {df['timestamp'].min().strftime('%Y-%m-%d')} to {df['timestamp'].max().strftime('%Y-%m-%d')}
- Overall Success Rate: {(df['status'] == 'success').mean() * 100:.1f}%
- Average Response Time: {df['total_latency_ms'].mean():.2f}ms
- P95 Response Time: {df['total_latency_ms'].quantile(0.95):.2f}ms

## Performance Metrics

### Latency Analysis
- Token Generation: {df['token_latency_ms'].mean():.2f}ms average
- Tool Execution: {df['tool_execution_ms'].mean():.2f}ms average
- Other Overhead: {df['total_latency_ms'].mean() - df['token_latency_ms'].mean() - df['tool_execution_ms'].mean():.2f}ms

### Throughput Metrics
- Requests per Hour: {len(df) / ((df['timestamp'].max() - df['timestamp'].min()).total_seconds() / 3600):.1f}
- Average Tokens per Request: {df['tokens_used'].mean():.0f}
- Tool Usage Rate: {(df['tools_used'] > 0).mean() * 100:.1f}%

## Issues Identified
"""
    
    if recommendations:
        for rec in recommendations:
            report += f"\n### {rec['category']} (Priority: {rec['priority']})\n"
            report += f"- Issue: {rec['issue']}\n"
            report += f"- Recommendation: {rec['recommendation']}\n\n"
    else:
        report += "\n✅ No critical performance issues detected.\n\n"
    
    report += f"""
## Recommendations Summary
- High Priority: {len([r for r in recommendations if r['priority'] == 'High'])} items
- Medium Priority: {len([r for r in recommendations if r['priority'] == 'Medium'])} items
- Low Priority: {len([r for r in recommendations if r['priority'] == 'Low'])} items

## Next Steps
1. Address high-priority issues immediately
2. Schedule medium-priority optimizations for next sprint
3. Monitor low-priority trends and plan future improvements
4. Set up automated alerts for key metrics

---
*Report generated by FunctionGemma Agent Performance Analysis Notebook*
"""
    
    return report

# Generate and save report
report = generate_performance_report(df, recommendations)

# Save to file
report_filename = f"performance_report_{datetime.now().strftime('%Y%m%d_%H%M%S')}.md"
with open(report_filename, 'w') as f:
    f.write(report)

print(f"\n📄 Report saved to: {report_filename}")
print("\n📊 Performance analysis completed successfully!")